# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV, KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_excel('RPDataset4.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540 entries, 0 to 1539
Data columns (total 5 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Explosive type                   1540 non-null   object 
 1   Explosive mass                   1540 non-null   float64
 2   Perpendicular standoff distance  1540 non-null   float64
 3   Incident angle                   1540 non-null   int64  
 4   Peak reflected pressure          1540 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 60.3+ KB


In [3]:
# convert categorical variable into dummy variables
dataset = pd.get_dummies(dataset, columns=['Explosive type'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540 entries, 0 to 1539
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Explosive mass                   1540 non-null   float64
 1   Perpendicular standoff distance  1540 non-null   float64
 2   Incident angle                   1540 non-null   int64  
 3   Peak reflected pressure          1540 non-null   float64
 4   Explosive type_Composition B     1540 non-null   uint8  
 5   Explosive type_TNT               1540 non-null   uint8  
dtypes: float64(3), int64(1), uint8(2)
memory usage: 51.3 KB


In [4]:
dataset.head()

,Explosive mass,Perpendicular standoff distance,Incident angle,Peak reflected pressure,Explosive type_Composition B,Explosive type_TNT
0,0.5,2.0,0,372.527,0,1
1,0.5,2.0,15,340.888,0,1
2,0.5,2.0,30,290.136,0,1
3,0.5,2.0,45,231.318,0,1
4,0.5,2.0,60,173.479,0,1


In [5]:
y = dataset[('Peak reflected pressure')]
X = dataset.drop('Peak reflected pressure', axis=1)
print(X.shape, y.shape)

(1540, 5) (1540,)


In [ ]:
print (X.head())

In [ ]:
y.head()

In [6]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 71)

In [8]:
X_val, X_test, y_val, y_test = train_test_split(X_test,
                                                y_test,
                                                test_size = 0.5,
                                                random_state = 71)

### Feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,0:2] = sc.fit_transform(X_train[:, 0:2])
print (X_train)

In [ ]:
X_test[:,0:2] = sc.transform(X_test[:, 0:2])
print (X_test)

In [ ]:
X_val[:,0:2] = sc.transform(X_val[:, 0:2])
print (X_val)

### S1 -Hyperparameter tuning - neurons, activation function

In [73]:
from tensorflow.keras.layers import Dropout
# Set seed for NumPy
np.random.seed(54)

# Set seed for TensorFlow
tf.random.set_seed(54)

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam
def create_model1(last_layer_nodes, activation_func):
    model = Sequential()
    model.add(Dense(units = 50,  input_shape=(X_train.shape[1],), activation=activation_func))
    #model.add(Dropout(0.1))
    model.add(Dense(last_layer_nodes, activation=activation_func))
    #model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear'))
    optimizer1 = Adam(learning_rate=0.01)
    model.compile(optimizer = optimizer1, loss = 'mean_squared_error', metrics = ['mae'])
    return model

activation_func = ['relu', 'leaky_relu', 'softplus']
last_layer_nodes = [ 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250]

param_grid = dict( model__activation_func = activation_func,model__last_layer_nodes = last_layer_nodes)

##Wrap model into scikit-learn
model1 = KerasRegressor(model=create_model1, verbose=0, epochs = 100, batch_size = 50, random_state = 54)

In [74]:
kf = KFold(n_splits=5, shuffle=True, random_state=54)
grid1 = GridSearchCV(estimator = model1, param_grid= param_grid, n_jobs=-1, scoring = 'r2', cv=kf)
grid_result1 = grid1.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result1.best_score_, grid_result1.best_params_))
means = grid_result1.cv_results_['mean_test_score']
stds = grid_result1.cv_results_['std_test_score']
params = grid_result1.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.966233 using {'model__activation_func': 'softplus', 'model__last_layer_nodes': 240}
0.951139 (0.017617) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 50}
0.958039 (0.003363) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 60}
0.960362 (0.007333) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 70}
0.960693 (0.005830) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 80}
0.958709 (0.007823) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 90}
0.959726 (0.009271) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 100}
0.959961 (0.005092) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 110}
0.961138 (0.007601) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 120}
0.956575 (0.006156) with: {'model__activation_func': 'relu', 'model__last_layer_nodes': 130}
0.960229 (0.005745) with: {'model__activation_func': 'relu', 'model__last_la

In [75]:
pd.concat([pd.DataFrame(params), pd.DataFrame({'model__first_layer_nodes': [50] * len(params)}),pd.DataFrame(means, columns=['R2'])], axis =1)

,model__activation_func,model__last_layer_nodes,model__first_layer_nodes,R2
0,relu,50,50,0.951139
1,relu,60,50,0.958039
2,relu,70,50,0.960362
3,relu,80,50,0.960693
4,relu,90,50,0.958709
...,...,...,...,...
58,softplus,210,50,0.960543
59,softplus,220,50,0.964460
60,softplus,230,50,0.964531
61,softplus,240,50,0.966233


### Merge all files

In [76]:
from pathlib import Path

current_dir = Path.cwd()
redirect_dir = current_dir.parent.parent.parent

#Access input folder
input_dir = Path (redirect_dir/"1D_Tracers/Hyperparameter tuning summary_ANN/Reflected_pressure_hyperparameter_tuning4")
print ("1",input_dir)

# Output Excel file
output_excel_file = Path(input_dir/"S1_summary.xlsx")

# List to store DataFrames from CSV files
dfs = []

# Loop through CSV files in the directory
for csv_file in input_dir.glob('*.csv'):
    # Read CSV file into a DataFrame and append to the list
    df = pd.read_csv(csv_file, header = None)
    dfs.append(df)

print (dfs)

# Concatenate DataFrames in the list along rows
merged_df = pd.concat(dfs, axis = 0,ignore_index=False)

# Write the merged DataFrame to an Excel file
merged_df.to_excel(output_excel_file, index=False)

1 G:\Chamodi\1D_Tracers\Hyperparameter tuning summary_ANN\Reflected_pressure_hyperparameter_tuning4
[     0         1    2    3         4
0    0      relu   50  100  0.962207
1    1      relu   60  100  0.960829
2    2      relu   70  100  0.954493
3    3      relu   80  100  0.963898
4    4      relu   90  100  0.963209
..  ..       ...  ...  ...       ...
58  58  softplus  210  100  0.961156
59  59  softplus  220  100  0.959963
60  60  softplus  230  100  0.961195
61  61  softplus  240  100  0.961925
62  62  softplus  250  100  0.961607

[63 rows x 5 columns],      0         1    2    3         4
0    0      relu   50  110  0.955566
1    1      relu   60  110  0.963125
2    2      relu   70  110  0.951621
3    3      relu   80  110  0.963335
4    4      relu   90  110  0.964423
..  ..       ...  ...  ...       ...
58  58  softplus  210  110  0.961259
59  59  softplus  220  110  0.960277
60  60  softplus  230  110  0.961853
61  61  softplus  240  110  0.962990
62  62  softplus  250  1

### S2 - Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [77]:
from tensorflow.keras.optimizers import RMSprop, Nadam
# Set seed for NumPy
np.random.seed(54)

# Set seed for TensorFlow
tf.random.set_seed(54)

# Function to create model, required for KerasClassifier
def create_model2():
 # create model
 model = Sequential()
 model.add(Dense(units=210, input_shape=(X_train.shape[1],), activation='relu'))
 #model.add(Dropout(0.1))
 model.add(Dense(units=210, activation='relu'))
 #model.add(Dropout(0.1))
 model.add(Dense(units=1, activation='linear'))

 return model

In [78]:
# create model
model2 = KerasRegressor(model=create_model2, verbose=0, random_state = 54, loss = 'mean_squared_error', metrics = ['mae'])

# define the grid search parameters
batch_size = [30,40,50]
optimizer = [Adam, Nadam, RMSprop]
learning_rate = [ 0.001,0.01, 0.1]
epochs = [500]

# gridsearch
param_grid2 = dict(batch_size=batch_size, optimizer=optimizer, optimizer__learning_rate = learning_rate, epochs = epochs)
kf = KFold(n_splits=5, shuffle=True, random_state=54)
grid2 = GridSearchCV(estimator=model2, param_grid=param_grid2, n_jobs=-1, scoring = 'r2', cv=kf)
grid_result2 = grid2.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result2.best_score_, grid_result2.best_params_))
means = grid_result2.cv_results_['mean_test_score']
stds = grid_result2.cv_results_['std_test_score']
params = grid_result2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.974955 using {'batch_size': 50, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.nadam.Nadam'>, 'optimizer__learning_rate': 0.01}
0.969364 (0.004384) with: {'batch_size': 30, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'optimizer__learning_rate': 0.001}
0.972086 (0.009706) with: {'batch_size': 30, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'optimizer__learning_rate': 0.01}
0.948065 (0.012732) with: {'batch_size': 30, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.adam.Adam'>, 'optimizer__learning_rate': 0.1}
0.972664 (0.004347) with: {'batch_size': 30, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.nadam.Nadam'>, 'optimizer__learning_rate': 0.001}
0.972204 (0.004602) with: {'batch_size': 30, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.nadam.Nadam'>, 'optimizer__learning_rate': 0.01}
0.957953 (0.010367) with: {'batch_size': 30, 'epochs': 500, 'optimizer': <class 'keras.src.optimizers.na

In [79]:
pd.concat([pd.DataFrame(params), pd.DataFrame(means, columns=['R2'])], axis =1)

,batch_size,epochs,optimizer,optimizer__learning_rate,R2
0,30,500,<class 'keras.src.optimizers.adam.Adam'>,0.001,0.969364
1,30,500,<class 'keras.src.optimizers.adam.Adam'>,0.010,0.972086
2,30,500,<class 'keras.src.optimizers.adam.Adam'>,0.100,0.948065
3,30,500,<class 'keras.src.optimizers.nadam.Nadam'>,0.001,0.972664
4,30,500,<class 'keras.src.optimizers.nadam.Nadam'>,0.010,0.972204
5,30,500,<class 'keras.src.optimizers.nadam.Nadam'>,0.100,0.957953
6,30,500,<class 'keras.src.optimizers.rmsprop.RMSprop'>,0.001,0.965841
7,30,500,<class 'keras.src.optimizers.rmsprop.RMSprop'>,0.010,0.971515
8,30,500,<class 'keras.src.optimizers.rmsprop.RMSprop'>,0.100,0.955796
9,40,500,<class 'keras.src.optimizers.adam.Adam'>,0.001,0.969755


In [ ]:
# Get the best hyperparameters from the grid search
best_params = grid_result2.best_params_
print (best_params)

In [ ]:
# Create the final model using the best hyperparameters
final_model = KerasRegressor(
    build_fn=create_model2,
    verbose=0,
    random_state=71,
    loss='mean_squared_error',
    metrics=['mae'],
    optimizer = Adam,
    optimizer__learning_rate = 0.1
    )

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   patience=50,
                   restore_best_weights = True)
# Train the final model on the entire training set
history = final_model.fit(X_train, y_train,
                validation_data = (X_val, y_val),
                callbacks=[es],
                epochs=best_params['epochs'],
                batch_size=best_params['batch_size'],
                verbose=1)

### Analyse learn history

In [ ]:
# let's see the training and validation accuracy by epoch
history_dict = history.history
loss_values = history_dict['loss'] # you can change this
val_loss_values = history_dict['val_loss'] # you can also change this
epochs = range(1, len(loss_values) + 1) # range of X (no. of epochs)
plt.plot(epochs, loss_values, 'blue', label='Train set loss')
plt.plot(epochs, val_loss_values, 'orange', label='Validation set loss')
#plt.title('Training and testing loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()

In [ ]:
loss_values1 = history_dict['mae'] # you can change this
val_loss_values1 = history_dict['val_mae'] # you can also change this
epochs = range(1, len(loss_values1) + 1) # range of X (no. of epochs)
plt.plot(epochs, loss_values1, 'blue', label='Train set MAE')
plt.plot(epochs, val_loss_values1, 'orange', label='Validation set MAE')
#plt.title('Training and testing MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()